<a href="https://colab.research.google.com/github/Kunal02880/Qunatum-phenomeno-/blob/main/cirac_zoller_Cnot_implementaion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qutip
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

# Parameters
#eta = 0.05  # Lamb-Dicke parameter
hbar = 1.0  # Reduced Planck's constant
Omega = 2 * np.pi * 1.0  # Rabi frequency
N = 2  # Truncation for motional states (n=0, n=1)

# Basis states
g = basis(2, 0)  # |g⟩ = |0⟩
e = basis(2, 1)  # |e⟩ = |1⟩
zero = basis(N, 0)  # Motional ground state |n=0⟩

# Operators
sigma_p = sigmap()  # Raising operator
sigma_m = sigmam()  # Lowering operator
a = destroy(N)  # Motional annihilation operator
adag = create(N)  # Motional creation operator

# Control qubit operators
sigma_p_control = tensor(sigma_p, qeye(2), qeye(N))
sigma_m_control = tensor(sigma_m, qeye(2), qeye(N))

# Target qubit operators
sigma_p_target = tensor(qeye(2), sigma_p, qeye(N))
sigma_m_target = tensor(qeye(2), sigma_m, qeye(N))

# Motional operators
a_op = tensor(qeye(2), qeye(2), a)
adag_op = tensor(qeye(2), qeye(2), adag)

# Hamiltonians
H_sideband_control = 0.5 * Omega * (sigma_p_control * a_op + sigma_m_control * adag_op)
H_carrier_target = Omega / 2 * (sigma_p_target + sigma_m_target)

# Input states
input_states = {
    "00": tensor(g, g, zero),
    "01": tensor(g, e, zero),
    "10": tensor(e, g, zero),
    "11": tensor(e, e, zero),
}

# Define computational basis for measurement
basis_states = [
    tensor(g, g, zero),  # |00⟩
    tensor(g, e, zero),  # |01⟩
    tensor(e, g, zero),  # |10⟩
    tensor(e, e, zero),  # |11⟩
]

# Function to calculate probabilities
def calculate_probabilities(psi_final):
    probs = []
    for state in basis_states:
        proj = state * state.dag()
        prob = (psi_final.dag() * proj * psi_final)
        probs.append(prob)
       # prob = (psi_final.dag() * proj * psi_final).full().item()
       # probs.append(prob)
    return probs

# Results storage
results = {}

# Apply CNOT gate for each input
for label, psi0 in input_states.items():
    # Step 1: Sideband transition on control qubit
    t_pi = np.pi / Omega
    U_sideband_control = (-1j * H_sideband_control * t_pi / hbar).expm()
    psi_after_step1 = U_sideband_control * psi0

    # Step 2: Carrier transition on target qubit
    U_carrier_target = (-1j * H_carrier_target * t_pi / hbar).expm()
    psi_after_step2 = U_carrier_target * psi_after_step1

    # Step 3: Reverse sideband transition on control qubit
    psi_final = U_sideband_control * psi_after_step2

    # Calculate probabilities
    probs = calculate_probabilities(psi_final)
    results[label] = probs

# Plot the results
input_labels = list(results.keys())
output_labels = ["|00⟩", "|01⟩", "|10⟩", "|11⟩"]

fig, ax = plt.subplots(figsize=(10, 6))
width = 0.2  # Bar width
x = np.arange(len(input_labels))

for i, output_label in enumerate(output_labels):
    probs = [results[input_label][i] for input_label in input_labels]
    ax.bar(x + i * width, probs, width, label=output_label)

ax.set_xticks(x + 1.5 * width)
ax.set_xticklabels(input_labels)
ax.set_xlabel("Input State")
ax.set_ylabel("Probability")
ax.set_title("CNOT Gate: Input vs Output Probabilities")
ax.legend(title="Output States")
plt.show()